# 【問題1】コンペティション内容の確認

## コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

# 何を学習し、何を予測するのか

 - クライアントの特徴項目を学習し、返済能力を予測する。

## どのようなファイルを作りKaggleに提出するか

- 特徴量同士の関係性を調べ、返済能力に関わりの深い項目を抜擢し、最終的な見解を加えて提出

## 提出されたものはどういった指標値で評価されるのか

- 提出物は 、予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価されます。

まずは単純な方法による ベースラインモデル　を作成します。精度の基準となるモデルです。
精度を高くする必要はありません。エラーなく実行でき、Kaggle側に推定値を提出できることを目指します。

# 【問題2】学習と検証

データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。
検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

データの分析
　　目的変数に関わりの深い項目をピックアップし、検証する
  　二値分類か多値分類か選択する
前処理
　　ピックアップした項目を加工する
    訓練用にデータを分割する
学習方法を選び、必要なら標準化をする
　　検証データをメソッドを使い推定する
　　検証データの特徴量を入力して得られた推定値と、検証データの正解値の近さを計算し、学習したモデルを評価します
必要に応じて可視化する
ROC曲線を作成
　　予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価される
テストデータに対して推定を行い、Kaggleに提出を行う

In [1]:
#主要なものをインポートする
import missingno as msno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings #ワーニング関連のモジュール？
warnings.filterwarnings('ignore') #ワーニングが消える？


df = pd.read_csv("application_train.csv")
#検証
df
#ROC曲線を描くには、真陽性率（TPR）と偽陽性率（FPR）のみが必要です
#TPRは、テスト中に利用可能なすべての陽性サンプルの中で発生する正しい陽性結果の数を定義します
#FPRは、テスト中に利用可能なすべての陰性サンプルの中で発生する誤った陽性結果の数を

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## データフレームの情報

In [2]:
#情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [3]:
#大きさ
df.shape

(307511, 122)

In [4]:
#確認
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 前処理

AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICEを比較特徴量として抜き出す。

In [5]:
#抜き出し
df_selected = df.loc[:, ["AMT_INCOME_TOTAL", "AMT_CREDIT","TARGET"]]#, "AMT_GOODS_PRICE"
df_selected.head()

,AMT_INCOME_TOTAL,AMT_CREDIT,TARGET
0,202500.0,406597.5,1
1,270000.0,1293502.5,0
2,67500.0,135000.0,0
3,135000.0,312682.5,0
4,121500.0,513000.0,0


欠損値のある行を削除

In [6]:
#削除
df_selected = df_selected.dropna(how='any')
df_selected.head()
df_selected.shape

(307511, 3)

確認

In [7]:
#確認
missing_data = pd.DataFrame(df_selected.isnull().sum())
missing_data

,0
AMT_INCOME_TOTAL,0
AMT_CREDIT,0
TARGET,0


## 学習モデル

In [8]:
# sklearnのから各種モデルをimportする
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# データの前処理
# データを分割
X_train, X_test, y_train, y_test = train_test_split(df_selected.iloc[:, :-1], df_selected.loc[:, "TARGET"], test_size=0.25, random_state=42)

# データを確認
print("X_train.shape : {}, X_test.shape : {}".format(X_train.shape, X_test.shape))
print("y_train.shape : {}, y_test.shape : {}".format(y_train.shape, y_test.shape))


X_train.shape : (230633, 2), X_test.shape : (76878, 2)
y_train.shape : (230633,), y_test.shape : (76878,)


## 最近傍法

In [9]:
# sklearnから標準化をするモデルをインポートする
from sklearn.preprocessing import StandardScaler
# モデルをインスタンス化して、モデルの中にあるメソッドを使える状態にする。
scaler = StandardScaler()

# 標準化をする（訓練用データで.fitを行う）
scaler.fit(X_train)

# 標準化をする（訓練用、検証用双方で.transformを行う）
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [10]:
# sklearnから最近傍法をするモデルをインポートする
from sklearn.neighbors import KNeighborsClassifier

# モデルに引数を渡してインスタン化する
neigh_1 = KNeighborsClassifier(n_neighbors=1)
neigh_3 = KNeighborsClassifier(n_neighbors=3)
neigh_5 = KNeighborsClassifier(n_neighbors=5)

# 学習する（訓練用データを.fitする）
neigh_1.fit(X_train_transformed, y_train)
neigh_3.fit(X_train_transformed, y_train)
neigh_5.fit(X_train_transformed, y_train)

# 推定する（検証用データを.predictする）
y_pred_neigh_1 = neigh_1.predict(X_test_transformed)
y_pred_neigh_3 = neigh_3.predict(X_test_transformed)
y_pred_neigh_5 = neigh_5.predict(X_test_transformed)

In [11]:
# モデルmetrics をimportすればaccuracy等を一つずつインポートしなくても良い
from sklearn import metrics 

# 評価する関数を作成する
def evaluate(y_true, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion

print("1-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_1)))
print("------")

print("3-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_3)))
print("------")

print("5-nn")
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred_neigh_5)))

1-nn
acc : 0.8584640599391243
precision : 0.09555018972059331
recall : 0.08948473590696172
f1 : 0.09241804987905582
confusion matrix 
 [[65443  5244]
 [ 5637   554]]
------
3-nn
acc : 0.9009599625380473
precision : 0.1172673480365788
recall : 0.03521240510418349
f1 : 0.054161490683229806
confusion matrix 
 [[69046  1641]
 [ 5973   218]]
------
5-nn
acc : 0.9152553396290226
precision : 0.10294117647058823
recall : 0.006784041350347278
f1 : 0.01272920139415063
confusion matrix 
 [[70321   366]
 [ 6149    42]]


# 【問題3】テストデータに対する推定

In [12]:
df_1 = pd.read_csv("application_test.csv")

df_1

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [13]:
#テストデータ抜き出し
df_selected_1 = df_1.loc[:, ["AMT_INCOME_TOTAL", "AMT_CREDIT"]]#AMT_GOODS_PRICE
df_selected_1.head()

,AMT_INCOME_TOTAL,AMT_CREDIT
0,135000.0,568800.0
1,99000.0,222768.0
2,202500.0,663264.0
3,315000.0,1575000.0
4,180000.0,625500.0


In [14]:
y_pred_neigh_5 = neigh_5.predict(df_selected_1)
y_pred_neigh_5_proba = neigh_5.predict_proba(df_selected_1)
print(y_pred_neigh_5_proba)
print(y_pred_neigh_5_proba.shape)
print(type(y_pred_neigh_5_proba))

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
(48744, 2)
<class 'numpy.ndarray'>


In [15]:
answer = y_pred_neigh_5_proba[:,1]
print(answer)

[0. 0. 0. ... 0. 0. 0.]


In [16]:
df_selected_2 = df_1.loc[:, ["SK_ID_CURR"]]  #IDのカラムを抽出
df_selected_3 = pd.Series(answer) #NP配列をDFに返還
df_answer =pd.concat([df_selected_2,df_selected_3],axis = 1) #DF同士を結合
print(df_answer)

       SK_ID_CURR    0
0          100001  0.0
1          100005  0.0
2          100013  0.0
3          100028  0.0
4          100038  0.0
...           ...  ...
48739      456221  0.0
48740      456222  0.0
48741      456223  0.0
48742      456224  0.0
48743      456250  0.0

[48744 rows x 2 columns]


In [17]:
df_answer_1 = df_answer.rename(columns ={0:'TARGET'}) #0のカラムをTARGETに変更
print(df_answer_1)

       SK_ID_CURR  TARGET
0          100001     0.0
1          100005     0.0
2          100013     0.0
3          100028     0.0
4          100038     0.0
...           ...     ...
48739      456221     0.0
48740      456222     0.0
48741      456223     0.0
48742      456224     0.0
48743      456250     0.0

[48744 rows x 2 columns]


In [18]:
df_answer_1.to_csv('/Users/daisukesatou/diveintocode-ml/dic_sinyo.csv',index=False)
#INDEXを除いたDFをCSVに書き込む

In [19]:
# 落とし込むカレントを検索
import os

path = os.getcwd()

print(path)       

/Users/daisukesatou/diveintocode-ml


- # Kaggleに提出、エラー無し

# 【問題4】特徴量エンジニアリング
## 精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。
  - どの特徴量を使うか
  - どう前処理をするか
何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。

In [20]:
# 引数の値を変えてみる
y_pred_neigh_7 = neigh_3.predict(df_selected_1)
y_pred_neigh_7_proba = neigh_3.predict_proba(df_selected_1)

In [21]:
print(y_pred_neigh_7_proba)
print(y_pred_neigh_7_proba.shape)
print(type(y_pred_neigh_7_proba))

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
(48744, 2)
<class 'numpy.ndarray'>


In [22]:
#最近傍方の引数を変えてみる
answer_1 = y_pred_neigh_7_proba[:,1]
print(answer_1)

[0. 0. 0. ... 0. 0. 0.]


In [23]:
#問３と同じ処理
df_selected_2 = df_1.loc[:, ["SK_ID_CURR"]]
df_selected_3 = pd.Series(answer)
df_answer =pd.concat([df_selected_2,df_selected_3],axis = 1)
print(df_answer_1)

       SK_ID_CURR  TARGET
0          100001     0.0
1          100005     0.0
2          100013     0.0
3          100028     0.0
4          100038     0.0
...           ...     ...
48739      456221     0.0
48740      456222     0.0
48741      456223     0.0
48742      456224     0.0
48743      456250     0.0

[48744 rows x 2 columns]


-  # 引数の値を変えてみたが特に変化は無し

## 特徴量を変えてみる。

In [24]:
#"AMT_CREDIT"を"AMT_GOODS_PRICE"に変更
df_selected_2 = df_1.loc[:, ["AMT_INCOME_TOTAL","AMT_GOODS_PRICE"]]
df_selected_2.head()

,AMT_INCOME_TOTAL,AMT_GOODS_PRICE
0,135000.0,450000.0
1,99000.0,180000.0
2,202500.0,630000.0
3,315000.0,1575000.0
4,180000.0,625500.0


In [25]:
# 問３と同じ処理
y_pred_neigh_6 = neigh_5.predict(df_selected_2)
y_pred_neigh_6_proba = neigh_5.predict_proba(df_selected_2)
print(y_pred_neigh_6_proba)
print(y_pred_neigh_6_proba.shape)
print(type(y_pred_neigh_6_proba))

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
(48744, 2)
<class 'numpy.ndarray'>


In [26]:
answer_1= y_pred_neigh_6_proba[:,1]
print(answer_1)

[0. 0. 0. ... 0. 0. 0.]


In [27]:
df_selected_4 = pd.Series(answer_1)
df_selected_3 = df_1.loc[:, ["SK_ID_CURR"]]
df_answer_2 =pd.concat([df_selected_3,df_selected_4],axis = 1)
df_answer_3 = df_answer_2.rename(columns ={0:'TARGET'})
print(df_answer_3)

       SK_ID_CURR  TARGET
0          100001     0.0
1          100005     0.0
2          100013     0.0
3          100028     0.0
4          100038     0.0
...           ...     ...
48739      456221     0.0
48740      456222     0.0
48741      456223     0.0
48742      456224     0.0
48743      456250     0.0

[48744 rows x 2 columns]


-  # 特徴量を変えてみたが、ほぼ変化なし。

# 前処理の変更

## 学習方法を変えてみる

In [28]:
# データの前処理
# データを分割
X_train, X_test, y_train, y_test = train_test_split(df_selected.iloc[:, :-1], df_selected.loc[:, "TARGET"], test_size=0.25, random_state=42)

# データを確認
print("X_train.shape : {}, X_test.shape : {}".format(X_train.shape, X_test.shape))
print("y_train.shape : {}, y_test.shape : {}".format(y_train.shape, y_test.shape))


X_train.shape : (230633, 2), X_test.shape : (76878, 2)
y_train.shape : (230633,), y_test.shape : (76878,)


In [29]:
# importしたモデルをインスタンス化
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression()
svc = SVC()
tree = DecisionTreeClassifier()
r_forest_c = RandomForestClassifier()

In [30]:
# 評価する関数を作成する
def evaluate(y_true, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    confusion = metrics.confusion_matrix(y_test, y_pred)
    return acc, precision, recall, f1, confusion


In [31]:
#ロジスティック回帰
print("Logistic Regression")
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred)))

Logistic Regression
acc : 0.9194698093082546
precision : 0.0
recall : 0.0
f1 : 0.0
confusion matrix 
 [[70687     0]
 [ 6191     0]]


In [32]:
#検証
y_pred_lr = lr.predict(df_selected_2)
y_pred_lr_proba = lr.predict_proba(df_selected_2)
answer_2= y_pred_lr_proba[:,1]
print(answer_2)

[0.12197004 0.21851743 0.05171551 ... 0.07275167 0.0500477  0.14603286]


- # ロジスティック回帰による学習ではわずかながら変化があった。

In [33]:
#決定木
print("Decision Tree")
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred)))

Decision Tree
acc : 0.906800385025625
precision : 0.0865874363327674
recall : 0.016475528993700535
f1 : 0.027683539150495318
confusion matrix 
 [[69611  1076]
 [ 6089   102]]


In [34]:
y_pred_tree = tree.predict(df_selected_2)
y_pred_tree_proba = tree.predict_proba(df_selected_2)
answer_3= y_pred_tree_proba[:,1]
print(answer_3)

[0.13355049 0.10769231 0.03333333 ... 0.07407407 0.09045226 0.05408516]


- # 決定木による学習でもわずかながら変化があった。

In [35]:
#ランダムフォレスト
print("Random Forest")
r_forest_c.fit(X_train, y_train)
y_pred = r_forest_c.predict(X_test)
print("acc : {}\nprecision : {}\nrecall : {}\nf1 : {}\nconfusion matrix \n {}".format(*evaluate(y_test, y_pred)))

Random Forest
acc : 0.9089596503551081
precision : 0.09109311740890688
recall : 0.014537231465029882
f1 : 0.025073129962390302
confusion matrix 
 [[69789   898]
 [ 6101    90]]


In [36]:
y_pred_forest = r_forest_c.predict(df_selected_2)
y_pred_forest_proba = r_forest_c.predict_proba(df_selected_2)
answer_4= y_pred_forest_proba[:,1]
print(answer_4)

[0.13486872 0.10604242 0.02952028 ... 0.07360684 0.09156395 0.05444626]


- # ランダムフォレストの結果をみても微細ながら変化は出た。

## 以上　５項目を検証した結果になります。